In [ ]:
# Exp Data Analysis
# https://towardsdatascience.com/a-machine-learning-project-predicting-used-car-prices-efbc4d2a4998#eced

In [27]:
%matplotlib inline

import quandl
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from dateutil.parser import parse
from sklearn import metrics

In [28]:
dfX = pd.read_csv("portfolio.csv", sep='\t')
dfX.set_index('Date')

,AAPL,AMZN,EXC,NDAQ,RGLD,Portfolio Val
Date,,,,,,
2015-07-01,2500.000000,2500.000000,1000.000000,3000.000000,1000.000000,10000.000000
2015-07-02,2496.840442,2501.829031,1010.085093,3025.867378,1020.802621,10055.424566
2015-07-06,2488.151659,2492.283774,1006.618342,3003.079450,1038.902539,10029.035764
2015-07-07,2482.030016,2496.170466,1032.776552,3008.006569,1009.500410,10028.484013
2015-07-08,2420.418641,2456.046092,1027.418847,2947.649353,1011.711712,9863.244645
...,...,...,...,...,...,...
2017-12-22,3608.659917,6678.021903,1340.219995,4888.929875,1467.426587,17983.258276
2017-12-26,3517.108291,6726.033974,1333.348834,4891.499952,1485.905041,17953.896092
2017-12-27,3517.726883,6757.470450,1339.532879,4904.992859,1470.308547,17990.031618


In [29]:
dfX = dfX[['Portfolio Val']]
dfX['Portfolio Val'] = dfX['Portfolio Val'].astype(float)
dfX

,Portfolio Val
0,10000.000000
1,10055.424566
2,10029.035764
3,10028.484013
4,9863.244645
...,...
624,17983.258276
625,17953.896092
626,17990.031618
627,17974.392222


In [30]:
dfX['Rolling_mean_PV'] = dfX['Portfolio Val'].rolling(5).mean()
dfX['Rolling_std_PV'] = dfX['Portfolio Val'].rolling(5).std()
dfX['upper_band_PV'] = dfX['Rolling_mean_PV']+ (2*dfX['Rolling_std_PV'])
dfX['lower_band_PV'] = dfX['Rolling_mean_PV']- (2*dfX['Rolling_std_PV'])
dfX.fillna(dfX.mean())

,Portfolio Val,Rolling_mean_PV,Rolling_std_PV,upper_band_PV,lower_band_PV
0,10000.000000,13369.735506,104.566367,13578.868239,13160.602773
1,10055.424566,13369.735506,104.566367,13578.868239,13160.602773
2,10029.035764,13369.735506,104.566367,13578.868239,13160.602773
3,10028.484013,13369.735506,104.566367,13578.868239,13160.602773
4,9863.244645,9995.237797,76.346138,10147.930074,9842.545520
...,...,...,...,...,...
624,17983.258276,18034.986713,109.541511,18254.069735,17815.903691
625,17953.896092,17983.406597,50.021882,18083.450361,17883.362832
626,17990.031618,17967.594727,19.109936,18005.814600,17929.374854
627,17974.392222,17973.523356,14.224080,18001.971516,17945.075196


In [31]:
data = pd.read_csv("S&P500.csv")
data['Date']  # as a Series
date_array = data['Date'].values  # as a numpy array
d_list = []
for date in date_array:
    dt = parse(date)
    d_item = (str(dt.date()))    
    d_list.append(d_item)
dfDate = pd.DataFrame(d_list,columns=['Date'])
dfDate.set_index('Date')
dfSP500 = pd.read_csv("S&P500.csv")
dfSP500.set_index('Date')
dfSP500 = dfSP500.rename(columns={'Price':'SP500', 'Date':'ODate'})

for i, col in enumerate(dfSP500.columns):
    dfSP500.iloc[:, i] = dfSP500.iloc[:, i].str.replace(',', '')
    
dfSP500 = dfSP500.join(dfDate)
dfSP500 = dfSP500.dropna()
dfSP500 = dfSP500[['SP500']]
dfSP500['SP500'] = dfSP500['SP500'].astype(float)
dfSP500 = dfSP500[::-1]
dfSP500.reset_index(inplace=True, drop=True)

In [32]:
dfX = dfX.join(dfSP500)
dfX.dropna()

,Portfolio Val,Rolling_mean_PV,Rolling_std_PV,upper_band_PV,lower_band_PV,SP500
4,9863.244645,9995.237797,76.346138,10147.930074,9842.545520,2046.68
5,9924.323042,9980.102406,82.425353,10144.953111,9815.251701,2051.31
6,10107.138356,9990.445164,96.314065,10183.073294,9797.817034,2076.62
7,10244.621207,10033.562252,150.770029,10335.102310,9732.022195,2099.60
8,10319.947293,10091.854909,197.437864,10486.730637,9696.979180,2108.95
...,...,...,...,...,...,...
624,17983.258276,18034.986713,109.541511,18254.069735,17815.903691,2679.25
625,17953.896092,17983.406597,50.021882,18083.450361,17883.362832,2684.57
626,17990.031618,17967.594727,19.109936,18005.814600,17929.374854,2683.34
627,17974.392222,17973.523356,14.224080,18001.971516,17945.075196,2680.50


In [64]:
dfX.describe()

,Portfolio Val,Rolling_mean_PV,Rolling_std_PV,upper_band_PV,lower_band_PV,SP500
count,629.000000,625.000000,625.000000,625.000000,625.000000,629.000000
mean,13373.595923,13369.735506,104.566367,13578.868239,13160.602773,2223.049141
std,2352.693243,2335.808491,65.500885,2324.243397,2354.616353,207.102255
min,9500.286457,9839.985634,5.041378,10144.953111,9048.230215,1829.080000
25%,11281.620234,11271.397308,61.324932,11466.753995,11091.556441,2071.180000
50%,13302.268737,13303.927717,90.126445,13519.850888,13092.697880,2168.270000
75%,15365.576956,15363.311964,135.399861,15604.631598,15041.415126,2396.920000
max,18309.025416,18207.281931,528.000548,18504.266930,18027.006482,2690.160000


In [35]:
dfX[4:].to_csv('XY_var.csv', sep='\t', encoding='utf-8')

In [36]:
y = pd.read_csv("XY_var.csv", sep='\t')

In [65]:
y = y['Portfolio Val']

KeyError: 'Portfolio Val'

In [66]:
y.to_numpy()

array([2046.68, 2051.31, 2076.62, 2099.6 , 2108.95, 2107.4 , 2124.29,
       2126.64, 2128.28, 2119.21, 2114.15, 2102.15, 2079.65, 2067.64,
       2093.25, 2108.57, 2108.63, 2103.84, 2098.04, 2093.32, 2099.84,
       2083.56, 2077.57, 2104.18, 2084.07, 2086.05, 2083.39, 2091.54,
       2102.44, 2096.92, 2079.61, 2035.73, 1970.89, 1893.21, 1867.61,
       1940.51, 1987.66, 1988.87, 1972.18, 1913.85, 1948.86, 1951.13,
       1921.22, 1969.41, 1942.04, 1952.29, 1961.05, 1953.03, 1978.09,
       1995.31, 1990.2 , 1958.03, 1966.97, 1942.74, 1938.76, 1932.24,
       1931.34, 1881.77, 1884.09, 1920.03, 1923.82, 1951.36, 1987.05,
       1979.92, 1995.83, 2013.43, 2014.89, 2017.46, 2003.69, 1994.24,
       2023.86, 2033.11, 2033.66, 2030.77, 2018.94, 2052.51, 2075.15,
       2071.18, 2065.89, 2090.35, 2089.41, 2079.36, 2104.05, 2109.79,
       2102.31, 2099.93, 2099.2 , 2078.58, 2081.72, 2075.  , 2045.97,
       2023.04, 2053.19, 2050.44, 2083.58, 2081.24, 2089.17, 2086.59,
       2089.14, 2088

In [67]:
y.shape

(625,)

In [68]:
x = pd.read_csv("XY_var.csv", sep='\t')

In [69]:
x = x[['SP500', 'Rolling_mean_PV', 'Rolling_std_PV', 'upper_band_PV', 'lower_band_PV']]

In [70]:
x = x.to_numpy()

In [71]:
x.shape

(625, 5)

In [72]:
x

array([[2.04668000e+03, 9.99523780e+03, 7.63461385e+01, 1.01479301e+04,
        9.84254552e+03],
       [2.05131000e+03, 9.98010241e+03, 8.24253526e+01, 1.01449531e+04,
        9.81525170e+03],
       [2.07662000e+03, 9.99044516e+03, 9.63140651e+01, 1.01830733e+04,
        9.79781703e+03],
       ...,
       [2.68334000e+03, 1.79675947e+04, 1.91099364e+01, 1.80058146e+04,
        1.79293749e+04],
       [2.68050000e+03, 1.79735234e+04, 1.42240800e+01, 1.80019715e+04,
        1.79450752e+04],
       [2.68262000e+03, 1.79516035e+04, 5.49079611e+01, 1.80614194e+04,
        1.78417876e+04]])

In [73]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20)

# Decision Tree Model

In [74]:
from sklearn.tree import DecisionTreeRegressor
regressor = DecisionTreeRegressor()
regressor.fit(x_train, y_train)

DecisionTreeRegressor()

In [75]:
y_pred = regressor.predict(x_test)

In [76]:
df=pd.DataFrame({'Actual':y_test, 'Predicted':y_pred})
df

,Actual,Predicted
109,2052.23,2053.19
430,2348.45,2349.25
9,2119.21,2119.12
521,2476.35,2477.83
78,2065.89,2066.13
...,...,...
427,2378.25,2373.47
207,2063.37,2063.95
121,2078.36,2078.58
462,2399.38,2399.63


In [77]:
from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 1.4263199999999907
Mean Squared Error: 5.425775199999941
Root Mean Squared Error: 2.3293293455413173


# Random Forest Regressor

In [90]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.50)

In [91]:
model = RandomForestRegressor(random_state=0).fit(x_train, y_train)
model.score(x_train, y_train), model.score(x_test, y_test)

(0.9999558922219778, 0.9993956400286814)

In [92]:
y_pred=model.predict(x_test)

In [93]:
df=pd.DataFrame({'Actual':y_test, 'Predicted':y_pred})
df

,Actual,Predicted
169,1989.26,1991.0000
525,2480.91,2477.6176
595,2578.87,2580.8502
352,2201.72,2201.8870
509,2459.27,2461.9923
...,...,...
294,2186.48,2186.0198
496,2439.07,2438.7043
142,1940.24,1943.0852
494,2434.50,2432.9520


In [94]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 2.711342172523603
Mean Squared Error: 25.020412882838976
Root Mean Squared Error: 5.00204087176814


In [95]:
bagging = RandomForestRegressor(max_features=None)# or we can pass max_features = len(X_train.columns) = 13
bagging.fit(x_train,y_train)
print('Training score ',bagging.score(x_train,y_train))
print('Test Error ',bagging.score(x_test,y_test))

Training score  0.9999449341312338
Test Error  0.9993901114359847


In [96]:
rf = RandomForestRegressor(max_features = 1) # same as book
rf.fit(x_train,y_train)
print('Training score ',rf.score(x_train,y_train))
print('Test Error ',rf.score(x_test,y_test))

Training score  0.9986877245448326
Test Error  0.990776030877684


In [111]:
feat_importances = pd.Series(rf.feature_importances_, index=dfX.head()[1:])
feat_importances.nlargest(25).plot(kind='barh',figsize=(10,10))

ValueError: Index data must be 1-dimensional

In [104]:
x_test

array([[ 1989.26      , 11219.48467335,    53.93687186, 11327.35841706,
        11111.61092963],
       [ 2480.91      , 16421.49289827,    56.7377955 , 16534.96848927,
        16308.01730727],
       [ 2578.87      , 17743.11863381,    82.37891988, 17907.87647357,
        17578.36079406],
       ...,
       [ 1940.24      , 10410.88127533,   177.27491282, 10765.43110096,
        10056.3314497 ],
       [ 2434.5       , 15699.96087712,    47.87154359, 15795.7039643 ,
        15604.21778995],
       [ 2159.73      , 13611.28206312,   105.59847313, 13822.47900939,
        13400.08511685]])